<a href="https://colab.research.google.com/github/glaucio-rocha/ciencia_de_dados/blob/main/MVP/MVP_Sprint_III_03_Consultas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install duckdb duckdb-engine jupysql altair


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.2 MB/s eta 0:00:00


### Configuração do ambiente

In [4]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

import pandas as pd
import geopandas as gpd
pd.options.display.max_rows = 50
pd.options.display.max_columns = None

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import altair as alt

import duckdb
%sql duckdb:///:memory:


### Preparação das tabelas
As tabelas são conectadas diretamente aos arquivos colunares .parquet publicados no repositório S3 da Backblaze

In [ ]:
%%sql

install httpfs;

DROP TABLE IF EXISTS fato_despesa;
DROP TABLE IF EXISTS dim_municipios;
DROP TABLE IF EXISTS dim_candidatos;

CREATE TABLE fato_despesa as
    SELECT * FROM 'https://gr-pucrio-mvp3-files.s3.us-west-001.backblazeb2.com/modelo/fato_despesa.parquet';

CREATE TABLE dim_candidatos as
    SELECT * FROM 'https://gr-pucrio-mvp3-files.s3.us-west-001.backblazeb2.com/modelo/dim_candidatos.parquet';

CREATE TABLE dim_municipios as
    SELECT * FROM 'https://gr-pucrio-mvp3-files.s3.us-west-001.backblazeb2.com/modelo/dim_municipios.parquet';






### Transformação de dados
Unifica as informações de municipios e uma unica tabela.
1. Aos dados de variação do PIB é adicionada a informação se o município pertence ao semiárido (transformando o código em SIM/NÃO)
2. A tabela é limitada aos municípios do estado de Sergipe
3. Os dados da geometria dos municípios foram mantidos na tabela

In [ ]:
%%sql

SELECT DS_SIT_TOT_TURNO, count(*) as total
  FROM 'https://gr-pucrio-mvp3-files.s3.us-west-001.backblazeb2.com/modelo/dim_candidatos.parquet'
  group by DS_SIT_TOT_TURNO



In [7]:
%%sql

SELECT  fd.SG_PARTIDO,sum(VR_DESPESA_CONTRATADA) as despesa_eleito
 from fato_despesa fd, dim_candidatos dc
 where fd.NR_CANDIDATO = dc.NR_CANDIDATO
  AND dc.DS_SIT_TOT_TURNO = 'ELEITO'
  group by fd.SG_PARTIDO
  order by despesa_eleito desc




,SG_PARTIDO,despesa_eleito
0,PSD,54742105.36
1,MDB,24199488.81
2,PSC,22205364.21
3,PDT,19070594.36
4,PT,18193792.08
5,PP,14673654.24
6,PL,13848409.28
7,DEM,7844600.22
8,CIDADANIA,6886662.20
9,PSB,2446052.28


### Investigando a despesa por partido em cada cidade
Nesta consulta foi utilizada a caracteristica analítica do banco de dados criando uma tabela pivot com os valores presentes na tabela de despesas.

A declaração PIVOT permite que valores distintos dentro de uma coluna sejam separados em suas próprias colunas. Os valores dentro dessas novas colunas são calculados usando uma função de agregação no subconjunto de linhas que correspondem a cada valor distinto.


In [29]:
%%sql r1 <<

select * from (
PIVOT (select SG_PARTIDO, NM_UE, VR_DESPESA_CONTRATADA
        from 'https://gr-pucrio-mvp3-files.s3.us-west-001.backblazeb2.com/modelo/fato_despesa.parquet')
 ON NM_UE
 USING sum(VR_DESPESA_CONTRATADA)
)

In [30]:
r1

,SG_PARTIDO,AMPARO DE SÃO FRANCISCO,AQUIDABÃ,ARACAJU,ARAUÁ,AREIA BRANCA,BARRA DOS COQUEIROS,BOQUIM,BREJO GRANDE,CAMPO DO BRITO,CANHOBA,CANINDÉ DE SÃO FRANCISCO,CAPELA,CARIRA,CARMÓPOLIS,CEDRO DE SÃO JOÃO,CRISTINÁPOLIS,CUMBE,DIVINA PASTORA,ESTÂNCIA,FEIRA NOVA,FREI PAULO,GARARU,GENERAL MAYNARD,GRACCHO CARDOSO,ILHA DAS FLORES,INDIAROBA,ITABAIANA,ITABAIANINHA,ITABI,ITAPORANGA D'AJUDA,JAPARATUBA,JAPOATÃ,LAGARTO,LARANJEIRAS,MACAMBIRA,MALHADA DOS BOIS,MALHADOR,MARUIM,MOITA BONITA,MONTE ALEGRE DE SERGIPE,MURIBECA,NEÓPOLIS,NOSSA SENHORA APARECIDA,NOSSA SENHORA DA GLÓRIA,NOSSA SENHORA DAS DORES,NOSSA SENHORA DE LOURDES,NOSSA SENHORA DO SOCORRO,PACATUBA,PEDRA MOLE,PEDRINHAS,PINHÃO,PIRAMBU,PORTO DA FOLHA,POÇO REDONDO,POÇO VERDE,PROPRIÁ,RIACHUELO,RIACHÃO DO DANTAS,RIBEIRÓPOLIS,ROSÁRIO DO CATETE,SALGADO,SANTA LUZIA DO ITANHY,SANTA ROSA DE LIMA,SANTANA DO SÃO FRANCISCO,SANTO AMARO DAS BROTAS,SIMÃO DIAS,SIRIRI,SÃO CRISTÓVÃO,SÃO DOMINGOS,SÃO FRANCISCO,SÃO MIGUEL DO ALEIXO,TELHA,TOBIAS BARRETO,TOMAR DO GERU,UMBAÚBA
0,PSD,12365.00,NaN,419715.77,76312.00,NaN,32454.70,154298.62,108960.60,9602.00,71426.75,250343.00,13824.00,125893.30,36168.45,16737.44,80000.00,74932.74,NaN,345903.62,22338.61,NaN,68653.50,44045.00,41153.26,52272.59,10299.80,61476.10,62351.99,69632.50,160999.57,22995.25,58726.35,37271.70,21022.90,114247.08,111584.49,435.00,13065.50,19060.55,69118.30,82195.72,59555.30,NaN,195720.72,158992.04,73448.65,23287.71,10538.90,47989.00,12811.75,NaN,42336.80,178600.25,46872.05,194616.43,42072.07,NaN,225142.35,20555.69,NaN,NaN,104934.51,31755.00,3678.50,59904.47,44929.05,NaN,216588.25,NaN,35013.16,NaN,84742.00,43613.50,9698.50,NaN
1,SOLIDARIEDADE,NaN,NaN,99555.70,101166.54,4980.00,138732.40,NaN,5113.00,NaN,NaN,0.00,33240.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7450.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7222.00,NaN,2585.00,710040.59,NaN,NaN,NaN,12730.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38929.00,NaN,NaN,NaN,17363.00,NaN,NaN,NaN,10000.00,NaN,NaN,NaN,NaN,2800.00,NaN,107404.65,10000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35040.50,39998.89,NaN,NaN,NaN,NaN
2,PSC,80621.00,NaN,225072.10,NaN,94650.52,NaN,NaN,NaN,NaN,73212.00,NaN,347862.70,97217.00,45500.90,NaN,NaN,96134.04,NaN,59010.85,NaN,NaN,NaN,0.00,NaN,121215.80,NaN,NaN,NaN,NaN,NaN,435392.89,1508.86,338797.86,NaN,NaN,NaN,NaN,NaN,NaN,300.00,NaN,NaN,NaN,NaN,4180.00,NaN,11620.70,104085.25,NaN,NaN,NaN,219587.02,NaN,30175.00,111294.50,NaN,NaN,NaN,146.00,NaN,NaN,NaN,89091.00,NaN,NaN,197919.68,NaN,236950.86,NaN,NaN,103056.00,NaN,NaN,NaN,NaN
3,REPUBLICANOS,NaN,95216.52,336581.67,NaN,NaN,15295.00,13350.00,NaN,NaN,NaN,18049.04,141830.83,52078.00,NaN,NaN,4784.10,NaN,NaN,30342.85,28842.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84512.85,21874.50,26013.00,NaN,16225.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93460.30,NaN,NaN,55916.93,69971.50,NaN,NaN,NaN,NaN,NaN,119161.03,NaN,20822.00,NaN,NaN,NaN,8916.00,25000.00,1149.00,NaN,NaN,63662.00,NaN,66543.53,29890.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CIDADANIA,NaN,3863.00,2830288.88,NaN,4250.00,56492.47,2104.00,NaN,NaN,NaN,NaN,7100.00,11485.64,NaN,NaN,NaN,NaN,NaN,91722.25,NaN,NaN,NaN,NaN,NaN,NaN,2010.00,NaN,7078.00,NaN,9200.00,4543.90,NaN,51087.48,117380.32,1436.00,NaN,NaN,3152.50,NaN,NaN,NaN,2480.50,6244.00,NaN,108227.62,NaN,383941.55,NaN,NaN,1000.35,NaN,NaN,4739.00,NaN,21073.08,NaN,NaN,2410.45,NaN,NaN,NaN,NaN,NaN,NaN,1504.00,964.00,NaN,36737.31,NaN,NaN,NaN,NaN,82139.00,0.00,23794.75
5,PT,NaN,14517.00,1912737.20,500.00,3180.00,380.00,9745.00,NaN,10494.00,NaN,173046.95,NaN,9569.00,NaN,5480.00,136287.90,NaN,NaN,93897.91,55230.71,16975.00,8756.00,89999.00,NaN,NaN,104548.73,56950.60,3613.00,0.00,30100.35,157106.00,0.00,8357.20,NaN,NaN,33597.93,16240.80,59483.75,NaN,4048.60,NaN,NaN,NaN,53044.72,35677.39,16746.39,239583.55,3000.00,NaN,34971.60,NaN,5064.00,6773.00,72832.69,35066.61,10305.00,10945.00,24732.70,41168.00,80300.80,71743.34,8364.00,NaN,NaN,57664.00,8292.01,NaN,26896.85,111591.34,NaN,NaN,NaN,NaN,83776.88,69186.35
6,DEM,NaN,NaN,1747915.67,18085.10,NaN,162775.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134243.90,

In [31]:

# create the chart
chart1 = alt.Chart(r1).mark_bar().encode(
    x=alt.X('SG_PARTIDO', type='nominal', title='Party'),
    y=alt.Y('ARACAJU', type='quantitative', title='Total Expenses'),
    color=alt.Color('SG_PARTIDO', type='nominal')
)


chart1.interactive()




alt.Chart(...)

In [32]:
%%sql r2 <<

select * from (
PIVOT (select fd.SG_PARTIDO, fd.VR_DESPESA_CONTRATADA, dc.DS_SIT_TOT_TURNO
        from 'https://gr-pucrio-mvp3-files.s3.us-west-001.backblazeb2.com/modelo/fato_despesa.parquet' fd,
            'https://gr-pucrio-mvp3-files.s3.us-west-001.backblazeb2.com/modelo/dim_candidatos.parquet' dc
            where fd.NR_CANDIDATO = dc.NR_CANDIDATO
      )
 ON DS_SIT_TOT_TURNO
 USING sum(VR_DESPESA_CONTRATADA)
)




In [33]:
# chart usando altair com o dataframe retornado pela query, SG_partido no eixo X e ELEITO no eixo y

alt.Chart(r2).mark_bar().encode(
    x=alt.X('SG_PARTIDO', type='nominal'),
    y=alt.Y('ELEITO', type='quantitative'),
    color=alt.Color('SG_PARTIDO', type='nominal')
).interactive()

alt.Chart(...)

In [34]:
# mostra dataframe sem apresentar notação cientifica
pd.options.display.float_format = '{:.2f}'.format
r2

,SG_PARTIDO,2º TURNO,ELEITO,ELEITO POR MÉDIA,ELEITO POR QP,NÃO ELEITO,SUPLENTE
0,PSD,NaN,54742105.36,6685831.99,20015474.62,51759476.88,48469559.86
1,SOLIDARIEDADE,NaN,1921277.52,128813.45,515651.88,7425412.97,2982876.73
2,PSC,NaN,22205364.21,689273.80,2148159.21,17885174.80,5488447.42
3,REPUBLICANOS,NaN,1649238.76,163159.59,1220054.60,11996606.93,4919492.57
4,CIDADANIA,10329993.30,6886662.20,283672.58,868551.48,65043982.53,3950766.79
5,PT,NaN,18193792.08,1532201.60,2821107.75,67667204.48,14353023.21
6,DEM,NaN,7844600.22,1279054.30,2862151.29,44930009.98,11761694.38
7,AVANTE,NaN,297052.78,125129.64,32070.05,3064302.20,708270.77
8,PP,NaN,14673654.24,1961992.89,4807237.49,23070781.89,17176519.19
9,PV,NaN,NaN,37332.65,148981.59,1977730.04,1155196.95


Grouping sets

In [53]:
%%sql r3 <<



SELECT SG_PARTIDO, DS_CARGO, COUNT(*) as eleitos
  FROM 'https://gr-pucrio-mvp3-files.s3.us-west-001.backblazeb2.com/modelo/dim_candidatos.parquet'
  where DS_SIT_TOT_TURNO  in ('ELEITO','ELEITO POR MÉDIA','ELEITO POR QP')
  GROUP BY GROUPING SETS ((SG_PARTIDO, DS_CARGO), SG_PARTIDO, DS_CARGO, ())
  having eleitos > 10



In [54]:
r3


,SG_PARTIDO,DS_CARGO,eleitos
0,PSD,Prefeito,20
1,PSD,Vereador,197
2,PP,Vereador,76
3,MDB,Vereador,86
4,CIDADANIA,Vereador,36
5,PL,Vereador,68
6,REPUBLICANOS,Vereador,31
7,PT,Vereador,60
8,DEM,Vereador,53
9,PSC,Vereador,66


Limpeza dos dados

In [17]:
%%sql

select SG_partido, VR_DESPESA_CONTRATADA
 from fato_despesa
 where isnan(VR_DESPESA_CONTRATADA)
  limit 50

,SG_PARTIDO,VR_DESPESA_CONTRATADA


In [56]:
%%sql mapa <<

SELECT *
from DIM_MUNICIPIOS

In [ ]:
mapa

,id_mun,nm_mun,variacao,semiarido,geometry
0,2800100.0,Amparo de São Francisco,-0.311015,SIM,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 62, 0, 0, 0, 247, ..."
1,2800209.0,Aquidabã,1.920057,SIM,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 54, 6, 0, 0, 74, 1..."
2,2801108.0,Canhoba,5.719373,SIM,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 42, 1, 0, 0, 220, ..."
3,2801207.0,Canindé de São Francisco,37.141519,SIM,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 66, 19, 0, 0, 188,..."
4,2801405.0,Carira,11.206600,SIM,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 24, 12, 0, 0, 118,..."
...,...,...,...,...,...
70,2806909.0,São Francisco,0.290266,NÃO,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 109, 3, 0, 0, 186,..."
71,2807204.0,Siriri,-6.731474,NÃO,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 24, 2, 0, 0, 208, ..."
72,2807501.0,Tomar do Geru,2.504160,NÃO,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 216, 29, 0, 0, 12,..."
73,2807600.0,Umbaúba,-1.352713,NÃO,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 184, 1, 0, 0, 52, ..."
